In [92]:
import sys
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ppandas import PDataFrame

# Data Processing

### Toronto Election Study (TES)


In [93]:
TES_df  = pd.read_stata('Toronto+Election+Study.dta')[['AGE','CPS5','CPS9']]

# Age
age_values = [17,23,28,33,38,43,48,53,58,63,114]
TES_df['AGE'] = pd.cut(TES_df['AGE'],age_values)
TES_df.astype({'AGE': str})

# Ward
TES_df.CPS5 = TES_df['CPS5'].str.replace(r"[\D]",'', regex=True)
TES_df.rename({'CPS5':'WARD44'},axis=1, inplace=True)
TES_df = TES_df[TES_df['WARD44'] !='']

# Candidate Vote - before election who they were leaning towards
mapDict = {'Doug Ford':'Doug Ford', 'Olivia Chow':'Olivia Chow', 'John Tory':'John Tory', 'Other':'Other', "Don't know or haven't decided": 'Unknown'}
TES_df.CPS9=TES_df.CPS9.map(mapDict)
TES_df.rename({'CPS9':'VOTE'},axis=1, inplace=True)
TES_df = TES_df[TES_df['VOTE'] !='Unknown']

age_marginal_df = pd.read_csv('census2011_age.csv')
ward_marginal_df = pd.read_csv('census2011_ward.csv', dtype={'WARD44': str})

In [94]:
age_marginal_pdf = PDataFrame.from_populational_data(independent_vars =["AGE"],data = age_marginal_df, num_of_records=2615090)
ward_marginal_pdf = PDataFrame.from_populational_data(["WARD44"],ward_marginal_df,2615090)

          Unnamed: 1
AGE                 
[17,19]     0.042431
[20,24]     0.085112
[25,29]     0.098280
[30,34]     0.093321
[35,39]     0.088327
[40,44]     0.091572
[45,49]     0.096310
[50,54]     0.088740
[55,59]     0.075400
[60,64]     0.065394
[65,114]    0.175115
        2011Census %
WARD44              
1           0.023447
10          0.024791
11          0.023856
12          0.020651
13          0.020370
14          0.020560
15          0.023728
16          0.020644
17          0.019315
18          0.017196
19          0.021888
2           0.021691
20          0.029292
21          0.018600
22          0.025053
23          0.033817
24          0.023844
25          0.022299
26          0.024821
27          0.030083
28          0.025462
29          0.017183
3           0.019973
30          0.020378
31          0.020485
32          0.021936
33          0.021854
34          0.022726
35          0.023879
36          0.020693
37          0.024605
38          0.025556
39          0

In [109]:
queryResults_ppandas = []
queryResults_TES = []
for seed in range(5):
    sample_pop = int(len(TES_df)*0.8)
    TES_df_subsample = TES_df.sample(sample_pop, random_state=seed)
    TES_pdf = PDataFrame(independent_vars = ['AGE','WARD44'], data =  TES_df_subsample)
    join_pdf = age_marginal_pdf.pjoin(TES_pdf,mismatches={"AGE":'numerical'})
    join_pdf = ward_marginal_pdf.pjoin(join_pdf)

    print('ppandas 2011 Census + TES(n = {}):'.format(join_pdf.num_of_records))
    queryResults= join_pdf.query(['VOTE'])
    queryResults['Probability(VOTE)'] = queryResults['Probability(VOTE)']*100.0
    queryResults_ppandas.append(queryResults)
    print(queryResults)

    print('TES(n = {})'.format(TES_pdf.num_of_records))
    queryResults= TES_pdf.query(['VOTE'])
    queryResults['Probability(VOTE)'] = queryResults['Probability(VOTE)']*100.0
    queryResults_TES.append(queryResults)
    print(queryResults)

ppandas 2011 Census + TES(n = 5232093):
          VOTE  Probability(VOTE)
0    Doug Ford          28.685391
1    John Tory          41.357111
2  Olivia Chow          25.943569
3        Other           4.013929
TES(n = 1913)
          VOTE  Probability(VOTE)
0    Doug Ford          25.711200
1    John Tory          47.705153
2  Olivia Chow          24.173699
3        Other           2.409948
ppandas 2011 Census + TES(n = 5232093):
          VOTE  Probability(VOTE)
0    Doug Ford          28.524143
1    John Tory          43.671837
2  Olivia Chow          24.617854
3        Other           3.186165
TES(n = 1913)
          VOTE  Probability(VOTE)
0    Doug Ford          26.820373
1    John Tory          49.811609
2  Olivia Chow          21.495675
3        Other           1.872342
ppandas 2011 Census + TES(n = 5232093):
          VOTE  Probability(VOTE)
0    Doug Ford          29.327076
1    John Tory          43.711778
2  Olivia Chow          23.522745
3        Other           3.438402
TE

In [110]:
queryResults_ppandas = pd.concat(queryResults_ppandas)
queryResults_TES = pd.concat(queryResults_TES)

In [127]:
res_ppandas = queryResults_ppandas.groupby(['VOTE'])['Probability(VOTE)'].agg(['mean','std']).sort_values('mean',ascending=False)
res_TES = queryResults_TES.groupby(['VOTE'])['Probability(VOTE)'].agg(['mean','std']).sort_values('mean',ascending=False)

In [132]:
actual_vote = pd.Series(data=[40.28, 33.73, 23.15, 2.84], index=['John Tory', 'Doug Ford', 'Olivia Chow', 'Other'])
res_ppandas['error'] = np.abs(res_ppandas['mean'] - actual_vote)
res_ppandas

,mean,std,error
VOTE,,,
John Tory,43.121427,1.103591,2.841427
Doug Ford,28.884506,0.314918,4.845494
Olivia Chow,24.460591,0.927616,1.310591
Other,3.533476,0.389694,0.693476


In [133]:
res_TES['error'] = np.abs(res_TES['mean'] - actual_vote)
res_TES

,mean,std,error
VOTE,,,
John Tory,48.792460,0.755199,8.512460
Doug Ford,26.460167,0.548977,7.269833
Olivia Chow,22.674795,1.027380,0.475205
Other,2.072578,0.216481,0.767422
